# About

This notebook shows examples of how to predict SLR using ML models or NBM methods at an individual HRRR grid point or across the HRRR's full domain.

Notes:



# Imports

In [1]:
import numpy as np
import pandas as pd
import os
import xarray as xr
import matplotlib.pyplot as plt
import sys

sys.path.append('../scripts/')
import hrrr_funcs

from scipy.spatial import KDTree
from cartopy.io.img_tiles import GoogleTiles

# Globals

In [2]:
# Contour levels and colors for SLR
SLR_INTV_LEVS = [
    2,
    4,
    6,
    8,
    10,
    12,
    14,
    16,
    18,
    20
]
SLR_FILL_COLORS = [
    '#348FC2',
    '#62B7AD',
    '#98DD94',
    '#DBFA7E',
    '#E7DF47',
    '#DFAB32',
    '#D47732',
    '#C54531',
    '#B62C31',
    '#950002'
]

# Subplots adjustments based on region
SUBPLOTS_ADJUST = {
    'WE': (0.958, 0.025, 0.035, 0.975, 0.061, 0.023),
    'CO': (0.925, 0.025, 0.2, 0.965, 0.061, 0.023),
    'WF': (0.9, 0.215, 0.225, 0.92, 0.061, 0.023),
    'NU': (0.925, 0.025, 0.18, 0.965, 0.061, 0.023),
    'NW': (0.945, 0.03, 0.12, 0.98, 0.061, 0.023),
    'SN': (0.905, 0.085, 0.2, 0.965, 0.061, 0.023),
    'SW': (0.925, 0.025, 0.145, 0.96, 0.061, 0.023),
    'IM': (0.925, 0.025, 0.175, 0.965, 0.061, 0.023),
    'CE': (0.953, 0.025, 0.04, 0.975, 0.061, 0.023),
    'EA': (0.953, 0.025, 0.04, 0.975, 0.061, 0.023),
    'GL': (0.87, 0.09, 0.15, 0.96, 0.061, 0.023),
    'NE': (0.925, 0.025, 0.165, 0.96, 0.061, 0.023),
    'NE_CASE': (0.925, 0.025, 0.165, 0.96, 0.061, 0.023),
    'MA': (0.91, 0.065, 0.145, 0.96, 0.061, 0.023)
}

# Colorbar positioning based on region
CB_POS = {
    'WE': (0.175, 0.0725),
    'CO': (0.9875, 0.145),
    'WF': (0.945, 0.215),
    'NU': (0.99, 0.145),
    'NW': (1, 0.15),
    'SN': (0.9875, 0.15),
    'SW': (0.9775, 0.145),
    'IM': (0.985, 0.145),
    'CE': (1, 0.15),
    'EA': (0.17, 0.175),
    'GL': (0.98, 0.15),
    'NE': (0.17, 0.1525),
    'NE_CASE': (0.165, 0.12),
    'MA': (0.98, 0.14)
}

# Read in HRRR sample data

In [4]:
datadir = '../data/'

yr, mn, dy, hr, fhr = 2023, 3, 14, 12, 16
init_time = pd.Timestamp(int(yr), int(mn), int(dy), int(hr))
valid_time = pd.Timestamp(int(yr), int(mn), int(dy), int(hr), int(fhr))

ds = hrrr_funcs.load_all_hrrr_vars(
    datadir,
    init_time,
    fhr
)
ds

<xarray.Dataset>
Dimensions:            (isobaricInhPa: 40, y: 1059, x: 1799)
Coordinates:
    time               datetime64[ns] ...
    step               timedelta64[ns] ...
  * isobaricInhPa      (isobaricInhPa) float64 1.013e+03 1e+03 ... 75.0 50.0
    latitude           (y, x) float64 ...
    longitude          (y, x) float64 -122.7 -122.7 -122.7 ... -60.95 -60.92
    valid_time         datetime64[ns] ...
    surface            float64 ...
    heightAboveGround  float64 ...
    meanSea            float64 ...
Dimensions without coordinates: y, x
Data variables: (12/16)
    t                  (isobaricInhPa, y, x) float32 ...
    u                  (isobaricInhPa, y, x) float32 ...
    v                  (isobaricInhPa, y, x) float32 ...
    q                  (isobaricInhPa, y, x) float32 ...
    w                  (isobaricInhPa, y, x) float32 ...
    r                  (isobaricInhPa, y, x) float32 ...
    ...                 ...
    d2m                (y, x) float32 ...
    u10                (y, x) float32 ...
    v10                (y, x) float32 ...
    orog               (y, x) float32 ...
    mslma              (y, x) float32 ...
    sp                 (y, x) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2025-01-29T14:36 GRIB to CDM+CF via cfgrib-0.9.1...